In [1]:
%config IPCompleter.greedy=True

In [6]:
import pandas as pd

df_train = pd.read_csv('train_kaggle.csv')
df_test = pd.read_csv('sample_solution.csv')
Y = df_train['Label'].values

## Load tsfresh

In [10]:
import pandas as pd
import numpy as np
from tqdm import tqdm
def load_features(col):
    return pd.read_parquet("filtered_features/filtered_feature{}.gzip".format(col))
def load_test_features(col):
    return pd.read_parquet("filtered_test_features/filtered_test_feature{}.gzip".format(col))

NUM_OF_COL = 9

XTrainT = []
XTestT = []
for col in tqdm(range(NUM_OF_COL)):
    tf = load_features(col)
    XTrainT.append(tf)
    tftest = load_test_features(col)
    tftest = tftest.replace([np.inf], np.finfo('float32').max).replace([np.inf, -np.inf], np.finfo('float32').min).fillna(0)
    XTestT.append(tftest)
XTrainT = np.concatenate(XTrainT,  axis=1)
XTestT = np.concatenate(XTestT,  axis=1)
print(XTrainT.shape)
print(XTestT.shape)

100%|███████████████████████████████████████████████████████████| 9/9 [00:04<00:00,  1.88it/s]


(18662, 5585)
(6051, 5585)


In [17]:
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier

clf = ExtraTreesClassifier(n_estimators=50)
# clf = GradientBoostingClassifier()
clf = clf.fit(XTrainT, Y)
modelSelection = SelectFromModel(clf, prefit=True, max_features=8000)
XTrainT = modelSelection.transform(XTrainT)
XTrainT.shape

(18662, 158)

In [18]:
XTestT = modelSelection.transform(XTestT)
XTestT.shape

(6051, 158)

## Load Normal

In [3]:
import numpy as np

dataframes = np.load('allData.npy', allow_pickle = True)
dataframes.shape

def load_test_dataframe(id):
    test_data = np.load("test/test/{}.npy".format(id))
    return pd.DataFrame(data=test_data)

testdatas = []
for id in df_test['Id']:
    dfi = load_test_dataframe(id)
    testdatas.append(dfi.values)

In [4]:
# feat = dataframes[0].shape[1]
feat = 12

In [5]:
from sklearn.preprocessing import OneHotEncoder

def one_hot_encoding(encoder, dfs, col):
    if encoder == None:
        data = set(dfs[0][:,col])
        for i in range(1,len(dfs)):
            df = dfs[i]
            data |= set(df[:,col])
        encoder = OneHotEncoder(handle_unknown='ignore')
        encoder = encoder.fit([[str(e)] for e in data])
    encodedData = []
    for df in dfs:
        encodedData.append(np.sum(encoder.transform([[str(e)] for e in df[:,col]]), axis=0))
    encodedData = np.stack(encodedData)    
    return encoder, encodedData

In [21]:
from tqdm import tqdm

encoders = []
encoder, XTrain = one_hot_encoding(None, dataframes, 0)
encoders.append(encoder)
for i in tqdm(range(1, feat)):
    encoder, train = one_hot_encoding(None, dataframes, i)
    encoders.append(encoder)
    XTrain = np.concatenate((XTrain, train), axis=1)
XTrain.shape

100%|█████████████████████████████████████████████████████████| 11/11 [05:10<00:00, 28.20s/it]


(18662, 60)

In [22]:
encoder, XTest = one_hot_encoding(None, testdatas, 0)
for i in tqdm(range(1, feat)):
    encoder, test = one_hot_encoding(None, testdatas, i)
    encoders.append(encoder)
    XTest = np.concatenate((XTest, test), axis=1)
XTest.shape

100%|█████████████████████████████████████████████████████████| 11/11 [01:27<00:00,  7.94s/it]


(6051, 60)

In [47]:
XTrain = np.squeeze(np.asarray(XTrain))
XTest = np.squeeze(np.asarray(XTest))

In [48]:
np.save('1_2_features_train.npy', XTrain)
np.save('1_2_features_test.npy', XTest)

In [19]:
XTrain = np.load('1_2_features_train.npy', allow_pickle = True)
XTest = np.load('1_2_features_test.npy', allow_pickle = True)
print(XTrain.shape)
print(XTest.shape)

(18662, 60)
(6051, 60)


In [20]:
XTrain = np.concatenate((XTrain, XTrainT), axis=1)
XTest = np.concatenate((XTest, XTestT), axis=1)
print(XTrain.shape)
print(XTest.shape)

(18662, 218)
(6051, 218)


In [26]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(XTrain, Y, test_size=0.2, random_state=42)
print(X_train.shape)
print(X_valid.shape)
print(y_train.shape)
print(y_valid.shape)

(14929, 218)
(3733, 218)
(14929,)
(3733,)


In [27]:
X_train

array([[ 34.        ,   3.        ,   0.        , ...,   0.75      ,
          0.56843273,   0.30357143],
       [ 44.        ,  11.        ,   0.        , ...,   0.315     ,
          1.6517424 ,   0.729     ],
       [119.        ,   6.        ,   0.        , ...,   0.88700565,
          0.83836551,   0.31638418],
       ...,
       [ 32.        ,   3.        ,   0.        , ...,   0.76470588,
          0.50791225,   0.29411765],
       [ 34.        ,   3.        ,   0.        , ...,   0.75      ,
          0.54240825,   0.30357143],
       [ 32.        ,   3.        ,   0.        , ...,   0.76470588,
          0.51875996,   0.29411765]])

In [28]:
import lightgbm as lgb

train_set = lgb.Dataset(X_train.astype(np.int32), y_train.astype(int))
valid_set = lgb.Dataset(X_valid.astype(np.int32), y_valid.astype(int))

from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier

clf = ExtraTreesClassifier(n_estimators=50)
clf = clf.fit(XTrain, Y)
modelSelection = SelectFromModel(clf, prefit=True, max_features=8000)
XTrain = modelSelection.transform(XTrain)
XTrain.shape

XTest = modelSelection.transform(XTest)
XTest.shape

In [29]:
from hyperopt import hp
from hyperopt import Trials
from hyperopt import fmin
from hyperopt import tpe
import lightgbm as lgb
from hyperopt import STATUS_OK

ITER = 50
STOP_ROUND = 5

# Create the dataset
def objective(params):
    """Objective function for Gradient Boosting Machine Hyperparameter Tuning"""
    params['num_leaves'] = int(params['num_leaves'])
    params['subsample_for_bin'] = int(params['subsample_for_bin'])
    params['min_child_samples'] = int(params['min_child_samples'])
    # Perform n_fold cross validation with hyperparameters
    # Use early stopping and evalute based on ROC AUC
    bst = lgb.train(params, train_set, ITER, valid_sets=valid_set, early_stopping_rounds=STOP_ROUND)
    bst.save_model('model.txt', num_iteration=bst.best_iteration)
  
    # Extract the best score
    best_score = bst.best_score['valid_0']['auc']
    
    # Loss must be minimized
    loss = 1 - best_score
    
    # Dictionary with information for evaluation
    return {'loss': loss, 'params': params, 'status': STATUS_OK}

# Define the search space
space = {
    'boosting_type': hp.choice('boosting_type', 
                                [
                                    'gbdt',
                                    'dart',
                                    'goss'
                                ]),
    'num_leaves': hp.quniform('num_leaves', 30, 150, 1),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.2)),
    'subsample_for_bin': hp.quniform('subsample_for_bin', 20000, 300000, 20000),
    'min_child_samples': hp.quniform('min_child_samples', 20, 500, 5),
    'reg_alpha': hp.uniform('reg_alpha', 0.0, 1.0),
    'reg_lambda': hp.uniform('reg_lambda', 0.0, 1.0),
    'colsample_bytree': hp.uniform('colsample_by_tree', 0.6, 1.0),
    'metric': 'auc'
}

# Trials object to track progress
bayes_trials = Trials()

MAX_EVALS = 500

# Optimize
best = fmin(fn = objective, space = space, algo = tpe.suggest, 
            max_evals = MAX_EVALS, trials = bayes_trials)
best

[1]	valid_0's auc: 0.948559                                                                   
Training until validation scores don't improve for 5 rounds                                   
[2]	valid_0's auc: 0.956241                                                                   
[3]	valid_0's auc: 0.957586                                                                   
[4]	valid_0's auc: 0.959617                                                                   
[5]	valid_0's auc: 0.959784                                                                   
[6]	valid_0's auc: 0.962841                                                                   
[7]	valid_0's auc: 0.964826                                                                   
[8]	valid_0's auc: 0.966302                                                                   
[9]	valid_0's auc: 0.933317                                                                   
[10]	valid_0's auc: 0.355954                      

[50]	valid_0's auc: 0.975549                                                                  
Did not meet early stopping. Best iteration is:                                               
[50]	valid_0's auc: 0.975549
[1]	valid_0's auc: 0.956148                                                                   
Training until validation scores don't improve for 5 rounds                                   
[2]	valid_0's auc: 0.967763                                                                   
[3]	valid_0's auc: 0.970978                                                                   
[4]	valid_0's auc: 0.973173                                                                   
[5]	valid_0's auc: 0.97415                                                                    
[6]	valid_0's auc: 0.974776                                                                   
[7]	valid_0's auc: 0.975426                                                                   
[8]	valid_0's auc: 0.

[3]	valid_0's auc: 0.960587                                                                   
[4]	valid_0's auc: 0.96028                                                                    
[5]	valid_0's auc: 0.960859                                                                   
[6]	valid_0's auc: 0.961153                                                                   
[7]	valid_0's auc: 0.961098                                                                   
[8]	valid_0's auc: 0.960789                                                                   
[9]	valid_0's auc: 0.961726                                                                   
[10]	valid_0's auc: 0.963501                                                                  
[11]	valid_0's auc: 0.9641                                                                    
[12]	valid_0's auc: 0.964883                                                                  
[13]	valid_0's auc: 0.965531                      

C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\callback.py:192: UserWarning: Early stopping is not available in dart mode
  warnings.warn('Early stopping is not available in dart mode')



[2]	valid_0's auc: 0.967975                                                                   
[3]	valid_0's auc: 0.97086                                                                    
[4]	valid_0's auc: 0.973283                                                                   
[5]	valid_0's auc: 0.973943                                                                   
[6]	valid_0's auc: 0.974782                                                                   
[7]	valid_0's auc: 0.975256                                                                   
[8]	valid_0's auc: 0.975389                                                                   
[9]	valid_0's auc: 0.975895                                                                   
[10]	valid_0's auc: 0.976087                                                                  
[11]	valid_0's auc: 0.976556                                                                  
[12]	valid_0's auc: 0.976585                      

...

[43]	valid_0's auc: 0.989124                                                                  
[44]	valid_0's auc: 0.989169                                                                  
[45]	valid_0's auc: 0.989247                                                                  
[46]	valid_0's auc: 0.989273                                                                  
[47]	valid_0's auc: 0.989206                                                                  
[48]	valid_0's auc: 0.989167                                                                  
[49]	valid_0's auc: 0.989096                                                                  
[50]	valid_0's auc: 0.989117                                                                  
[1]	valid_0's auc: 0.95133                                                                    
Training until validation scores don't improve for 5 rounds                                   
[2]	valid_0's auc: 0.962388                       

[25]	valid_0's auc: 0.978577                                                                  
[26]	valid_0's auc: 0.979091                                                                  
[27]	valid_0's auc: 0.979396                                                                  
[28]	valid_0's auc: 0.979593                                                                  
[29]	valid_0's auc: 0.980129                                                                  
[30]	valid_0's auc: 0.980442                                                                  
[31]	valid_0's auc: 0.980689                                                                  
[32]	valid_0's auc: 0.981023                                                                  
[33]	valid_0's auc: 0.981239                                                                  
[34]	valid_0's auc: 0.981391                                                                  
[35]	valid_0's auc: 0.981469                      

[12]	valid_0's auc: 0.966979                                                                  
[13]	valid_0's auc: 0.967641                                                                  
[14]	valid_0's auc: 0.967944                                                                  
[15]	valid_0's auc: 0.968142                                                                  
[16]	valid_0's auc: 0.968548                                                                  
[17]	valid_0's auc: 0.968889                                                                  
[18]	valid_0's auc: 0.969523                                                                  
[19]	valid_0's auc: 0.970201                                                                  
[20]	valid_0's auc: 0.970673                                                                  
[21]	valid_0's auc: 0.97124                                                                   
[22]	valid_0's auc: 0.971579                      

[45]	valid_0's auc: 0.984825                                                                  
[46]	valid_0's auc: 0.984893                                                                  
[47]	valid_0's auc: 0.985008                                                                  
[48]	valid_0's auc: 0.985116                                                                  
[49]	valid_0's auc: 0.985372                                                                  
[50]	valid_0's auc: 0.985553                                                                  
Did not meet early stopping. Best iteration is:                                               
[50]	valid_0's auc: 0.985553
100%|██████████████████████| 500/500 [14:53<00:00,  1.79s/it, best loss: 0.009359867997189686]


{'boosting_type': 0,
 'colsample_by_tree': 0.6372851567624551,
 'learning_rate': 0.1550933641538317,
 'min_child_samples': 35.0,
 'num_leaves': 112.0,
 'reg_alpha': 0.9330450552639515,
 'reg_lambda': 0.2070177166075471,
 'subsample_for_bin': 300000.0}

In [31]:
best['boosting_type'] = 'dart'
best['num_leaves'] = int(best['num_leaves'])
best['subsample_for_bin'] = int(best['subsample_for_bin'])
best['min_child_samples'] = int(best['min_child_samples'])
lgbModel = lgb.train(best, train_set, ITER, valid_sets=valid_set, early_stopping_rounds=STOP_ROUND)
lgbModel.save_model('model.txt', num_iteration=lgbModel.best_iteration)
YTest = lgbModel.predict(XTest)

In [32]:
from lightgbm import LGBMClassifier
model = LGBMClassifier(boosting_type=best['boosting_type'],
                        num_leaves=best['num_leaves'],
                        learning_rate=best['learning_rate'],
                        subsample_for_bin=best['subsample_for_bin'],
                        min_child_samples=best['min_child_samples'],
                        reg_alpha=best['reg_alpha'],
                        reg_lambda=best['reg_lambda'],
                        colsample_bytree=best['colsample_by_tree'])
model.fit(XTrain, Y)
probs = model.predict_proba(XTest, num_iteration=lgbModel.best_iteration)

In [21]:
from lightgbm import LGBMClassifier
model = LGBMClassifier()
model.fit(XTrain, Y)
probs = model.predict_proba(XTest)

In [33]:
YTest = probs[:, 0]

In [34]:
df_test['Predicted'] = YTest
df_test

,Id,Predicted
0,0,0.001814
1,1,0.013528
2,2,0.998203
3,3,0.032190
4,4,0.798199
5,5,0.945413
6,6,0.998372
7,7,0.867544
8,8,0.001662
9,9,0.003749


In [35]:
df_test.to_csv('test.csv', index=False)